In [1]:
import tensorflow as tf
import numpy as np

import os
from pinn import PINNModel, model1, model2
from data_gen import DataGenerator, real_u1, get_koefs

2024-10-17 22:43:13.106129: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-17 22:43:13.140098: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 22:43:13.180612: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-17 22:43:13.192769: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-17 22:43:13.222553: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# force gpu use
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

In [3]:
area_grid_size = (50, 50)
border_grid_size = (50, 50)
x_limits = (0, 2)
y_limits = (0, 2)

EPOCHS = 5000

# Define the learning rate schedule
boundaries = [5000, 10000, 50000]
values = [1e-3, 1e-4, 1e-5, 1e-6]

lr_sched = tf.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)

model = PINNModel(
    model2(), optm=tf.keras.optimizers.Adam(learning_rate=lr_sched))

dataGen = DataGenerator(x_limits, y_limits, model.predict, real_u1)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,297 (12.88 KB)

 Trainable params: 3,297 (12.88 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
koefs = get_koefs()

session_n = 2

data_set = dataGen.inner_pairs(area_grid_size), dataGen.border_pairs(border_grid_size)

for koef in koefs:
    path = f"../models/s{session_n}.model{koef}.weights.h5"
    if not os.path.exists(path):
        model.reset()
        model.fit(koef, *data_set, EPOCHS, EPRINT=EPOCHS)
        model.save(path)

Time past 28.268805503845215

Ef
